In [2]:
import ast
import pandas as pd
import datetime

In [3]:
def diff_dates(d1, d2):
    date1 = datetime.datetime.strptime(d1.replace(",","").strip(), '%Y-%m-%d')
    date2 = datetime.datetime.strptime(d2.replace(",","").strip(), '%Y-%m-%d')
    return abs(date2-date1).days
def timedatelinesplit(newKick, columnName):
    newKick[columnName+"_datetime"] = newKick[columnName].apply(lambda x: datetime.datetime.fromtimestamp(float(x)).strftime('%Y-%m-%d , %H:%M:%S, %A'))
    new = newKick[columnName+"_datetime"].str.split(",", n = 2, expand = True) 
    newKick[columnName+"_date"] = new[0]
    newKick[columnName+"_time"] = new[1]
    timeDayArray =  []
    for hours in new[1]:
        hours = hours.strip().split(":")

        timeOfDay = ""
        if int(hours[0]) in [12,13,14,15]:
            timeOfDay = "Afternoon"
            timeDayArray.append(timeOfDay)
        elif hours[0] in [16,17,18,19,20]:
            timeOfDay = "Evening"
            timeDayArray.append(timeOfDay)
        elif hours[0] in [20,21,22,23,24]:
            timeOfDay = "Night"
            timeDayArray.append(timeOfDay)
        else:
            timeOfDay = "Morning"
            timeDayArray.append(timeOfDay)
        
    newKick[columnName+"_moment"] = timeDayArray
    newKick[columnName+"_day"] = new[2]
    
    newKick = newKick.drop(columns = [columnName, columnName+"_datetime"])
    
    return newKick
def getDfValues(df,names):
    return df[names[0]], df[[names[1]]]
def stateReturn(state):
    if state == 'successful':
        return 1
    else:
        return 0

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
kick = pd.read_csv("Kickstarter001.csv")
# kick = pd.read_csv("/content/drive/My Drive/Kickstarter001.csv")

In [5]:
## Drop non needed columns
newKick = kick.drop(columns = ["is_backing","is_starred","static_usd_rate","urls",
                     "source_url","currency_trailing_code","id","slug", "current_currency",
                     "currency","profile","photo","permissions","created_at",
                     "usd_pledged","usd_type","converted_pledged_amount",
                     "currency_symbol","disable_communication","friends",
                     "fx_rate","is_starrable"
                     ])

## Get only for USA
newKick = newKick[newKick["country"] == "US"]
newKick = newKick.apply(lambda row: row[newKick['state'].isin(['successful','failed'])])

In [6]:
namesArray = []
slugArray = []
categoryNameSlugArray = []
for row in newKick["category"]:
    rowDict = ast.literal_eval(row)
    namesArray.append(rowDict["name"])
    
    slugArray.append(rowDict["slug"].split("/")[0])
    categoryNameSlugArray.append(rowDict["name"] + " "+rowDict["slug"].split("/")[0])
newKick["category_name"] = namesArray
newKick["category_slug"] = slugArray
newKick["category_name_slug"] = categoryNameSlugArray
newKick = newKick.drop(columns = ["category"])

In [7]:
creatorArray = []
creatorNameArray = []   
for row in newKick["creator"]:
    rowDict = str(row)
    rowArray = rowDict.split(":")
    rowOneArray = rowArray[1].split(",")
    creatorArray.append(rowOneArray[0])
    rowTwoArray = rowArray[2].split(",")
    creatorNameArray.append(rowTwoArray[0])
newKick["creator_id"] = creatorArray
newKick["creator_name"] = creatorNameArray

newKick = newKick.drop(columns = ["creator"])

locationArray = []
locationNameArray = []  
for row in newKick["location"]:
    if str(row) != 'nan': 
        rowDict = str(row)
        rowArray = rowDict.split(",")
        rowOneArray = rowArray[1].split(":")
        rowTwoArray = rowArray[9].split(":")
        
        locationArray.append(rowOneArray[1].replace('\"',"").strip() + " "+ rowTwoArray[1].replace('\"',"").strip())
    else:
        locationArray.append("help")

newKick["city_state"] = locationArray
newKick = newKick.drop(columns = ["location", "country"])    
newKick = timedatelinesplit(newKick, "deadline")
newKick = timedatelinesplit(newKick, "launched_at")
newKick["goal_pledged_diff"] = newKick["pledged"] - newKick["goal"]
newKick["duration_for_days"] = newKick[["deadline_date","launched_at_date"]].apply(lambda x: diff_dates(x[0],x[1]), axis = 1)
newKick["staff_pick"] = newKick["staff_pick"]


### EDA GroupBys
#category = newKick[newKick["state"]=="successful"].groupby("category_slug").count()
category = newKick[newKick["state"]=="successful"].groupby("category_slug").count()
cleanedUpKick = newKick.drop(columns=["backers_count","pledged","state_changed_at",
                      "deadline_date","deadline_time","launched_at_date", "creator_name" , "category_name","category_slug","goal_pledged_diff","spotlight"])
cols = ['goal', 'duration_for_days']
subset_df = newKick[cols]

#Standard Scaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
scaled_df = ss.fit_transform(subset_df)
scaled_df = pd.DataFrame(scaled_df, columns=cols)


# Category Slug
cleanedUpKick['staff_pick'] = cleanedUpKick['staff_pick'].astype(int)
cleanedUpKick['state'] = cleanedUpKick['state'].apply(lambda x:stateReturn(x))
cleanedWithKickTime = pd.get_dummies(cleanedUpKick, columns=["category_name_slug","city_state","launched_at_day","launched_at_moment"])
cleanedWithKickTime[["goal","duration_for_days"]] = scaled_df
cleanedWithKickTime.dropna(axis='rows',inplace=True)
cleanedWithKickTimeBlurb = cleanedWithKickTime['blurb']
cleanedWithKickTimeName =  cleanedWithKickTime['name']
X = cleanedWithKickTime.drop(columns=["state","blurb","deadline_moment","name","deadline_day","launched_at_time","creator_id"])
Y = cleanedWithKickTime["state"]

/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)

from sklearn import metrics
prediction_test = model.predict(X_test)
logistic_prediction_proba = model.predict_proba(X_test)[:,1]
print (metrics.accuracy_score(y_test, prediction_test))

## To get the weights of all the variables
weights = pd.Series(model.coef_[0],index=X.columns.values)
weightSorted = weights.sort_values(ascending = False)


0.7361963190184049


/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# GridSearchCV and LASSO Hyper Tuning

In [ ]:
# pip install sklearn

In [10]:
from sklearn.linear_model import LogisticRegression
import pickle
model = LogisticRegression(penalty='l1') ## LASSO 
result = model.fit(X_train, y_train)

from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(cv=None,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l1', tol=0.0001),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})
             
resultclf = clf.fit(X_train, y_train)

from sklearn import metrics
prediction_test = model.predict(X_test)
print (metrics.accuracy_score(y_test, prediction_test))
prediction_test_clf = clf.predict(X_test)
prediction_test_proba = clf.predict_proba(X_test)
print (metrics.accuracy_score(y_test, prediction_test_clf))

clf.feature_names = list(X_train.columns)
# predictor/
with open("logisticNeuralNet.pkl", "wb") as f:
    pickle.dump(clf, f)

## To get the weights of all the variables
weights = pd.Series(model.coef_[0],index=X.columns.values)
weightSorted = weights.sort_values(ascending = False)
weights_clf = pd.Series( clf.best_estimator_.coef_[0],index=X.columns.values)
weightSorted_clf = weights_clf.sort_values(ascending = False)

/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/line

0.7116564417177914
0.7208588957055214


/Users/opasina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Fine Tuned Parameters

In [19]:
weightSorted_clf.to_csv('finalPages.csv')

/Users/opasina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [15]:
print(weightSorted_clf.head(10))

city_state_Hilton Head Island SC         7.110524
city_state_Grand Rapids MI               6.045215
category_name_slug_Fiction publishing    5.972428
city_state_Fort Davis TX                 5.903726
city_state_Lithonia GA                   5.669309
city_state_Hinsdale IL                   5.586010
city_state_Princeton NJ                  5.543553
category_name_slug_Video Games games     5.528623
city_state_Dartmouth MA                  5.491518
category_name_slug_Crafts crafts         5.482881
dtype: float64


In [16]:
print(weightSorted_clf.tail(10))

category_name_slug_Comedy music            -4.647498
city_state_Yoakum TX                       -4.887002
category_name_slug_Childrenswear fashion   -4.919344
city_state_Hilo HI                         -4.959460
city_state_Tulum Quintana Roo              -4.989510
category_name_slug_Live Games games        -5.777723
category_name_slug_Farms food              -5.970017
category_name_slug_Food Trucks food        -6.430207
category_name_slug_Web journalism          -7.200177
category_name_slug_Mobile Games games      -7.348971
dtype: float64


# Run on Colab

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import (Dense, Embedding, Reshape, Activation, 
                          SimpleRNN, LSTM, Convolution1D, 
                          MaxPooling1D, Dropout, Bidirectional,SpatialDropout1D)
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

Using TensorFlow backend.


# NEURAL NETWORK LSTM METHODS

In [ ]:
maxlen = 100
EMBEDDING_DIM = 100

In [ ]:
import os
import numpy as np
def create_glove_embedded():
    embeddings_index = {}
    GLOVE = '/content/drive/My Drive/glove.6B.100d.txt'
    f = open(GLOVE)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
return embeddings_index

In [ ]:
def embedded_matrix(embeddings_index,word_index,EMBEDDING_DIM):
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)

        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
def get_tokenizer_word_index(cleanedWithKickTimeBlurb):
    tokenizer = Tokenizer(num_words = len(cleanedWithKickTimeBlurb), filters="""!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',""")
    tokenizer.fit_on_texts(cleanedWithKickTimeBlurb.values)
    word_index = tokenizer.word_index
    return word_index

In [2]:
!pip install dill

    100% |████████████████████████████████| 153kB 5.1MB/s ta 0:00:01
  Stored in directory: /Users/opasina/Library/Caches/pip/wheels/c9/de/a4/a91eec4eea652104d8c81b633f32ead5eb57d1b294eab24167
Successfully built dill


In [4]:
import dill
import pickle
def convertTextToWord(word):
    tokenizer = Tokenizer(num_words = len(word.split()), filters="""!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',""")
    tokenizer.fit_on_texts(word)
    word_index = tokenizer.word_index
    X_blurb = tokenizer.texts_to_sequences(word)
    X_blurb = pad_sequences(X_blurb, maxlen=maxlen)
    
    return X_blurb
# ser = dill.dumps(convertTextToWord)

#Pickle model
with open("converTextToWord.pkl", "wb") as f:
    pickle.dump(convertTextToWord, f)

In [ ]:
def neuralNetworkForText(cleanedWithKickTimeBlurb,embedding_matrix,maxlen,lstm_param):
    tokenizer = Tokenizer(num_words = len(cleanedWithKickTimeBlurb), filters="""!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',""")
    tokenizer.fit_on_texts(cleanedWithKickTimeBlurb.values)
    word_index = tokenizer.word_index
    X_blurb = tokenizer.texts_to_sequences(cleanedWithKickTimeBlurb)
    X_blurb = pad_sequences(X_blurb, maxlen=maxlen)
    X_train_blurb, X_test_blurb, y_train_blurb, y_test_blurb = train_test_split(X_blurb,Y, test_size = 0.20)

    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,weights=[embedding_matrix],
                              trainable=False))
    # model.add(Dropout(0.2))
    model.add(LSTM(lstm_param, dropout=0.2, recurrent_dropout=0.2))
    # model.add(Dropout(0.20))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

    model.summary()
    epochs = 20
    batch_size = 32
    model.fit(X_train_blurb, y_train_blurb, batch_size=batch_size, epochs=epochs, 
                validation_data=(X_test_blurb, y_test_blurb),
           callbacks=[EarlyStopping(patience=10, verbose=1,restore_best_weights=True),
                      ReduceLROnPlateau(factor=.5, patience=3, verbose=1)])
    hidden_features_blurb = model.predict(X_test_blurb)
    
    hidden_features_result = model.predict_proba(X_test_blurb)
    
    hidden_features_array = [x[0] for x in hidden_features_result]


    return model, hidden_features_array

# NEURAL NETWORK KICKSTARTER BLURB

In [ ]:
embeddings_index = create_glove_embedded()
word_index = get_tokenizer_word_index(cleanedWithKickTimeBlurb)
embedding_matrix = embedded_matrix(embeddings_index,word_index,EMBEDDING_DIM)
predictblurbModelProb = neuralNetworkForText(cleanedWithKickTimeBlurb,embedding_matrix,maxlen,50)
predictblurbModel = predictblurbModelProb[0]
predictblurbProb = predictblurbModelProb[1]

#Pickle model
with open("blurbNeuralNet.pkl", "wb") as f:
    pickle.dump(predictblurbModel, f)

# NEURAL NETWORK KICKSTARTER NAME

In [ ]:
word_index_name = get_tokenizer_word_index(cleanedWithKickTimeName)
embedding_matrix_name = embedded_matrix(embeddings_index,word_index_name,EMBEDDING_DIM)
predictNameModelProb = neuralNetworkForText(cleanedWithKickTimeName,embedding_matrix_name,maxlen,40)
predictNameModel = predictNameModelProb[0]
predictNameProb = predictNameModelProb[1]

#Pickle model
with open("nameNeuralNet.pkl", "wb") as f:
    pickle.dump(predictNameModel, f)

## CREATING SINGLE PREDICTION FROM MULTIPLE PREDICTIONS (ENSEMBLING- STACKING)

In [ ]:
probToResult = pd.DataFrame(
    {'predict_name': predictNameProb,
     'predict_blurb': predictblurbProb,
     'logistic_prediction' : prediction_test_proba[:,1],
     'result': y_test
    })

In [ ]:
# cols = ['predict_name', 'predict_blurb','logistic_prediction']
# subset_df = probToResult[cols]
# #Standard Scaler
# ss = StandardScaler()
# scaled_df_prob = ss.fit_transform(subset_df)
# scaled_df_prob = pd.DataFrame(scaled_df_prob, columns=cols)

In [ ]:
X = scaled_df_prob[['predict_name','predict_blurb','logistic_prediction']]
Y = probToResult['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)

# save the model to disk
with open("lrResult.pkl", "wb") as f:
    pickle.dump(model, f)